In [51]:
import subprocess
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams['font.sans-serif'] = 'Arial'
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
from matplotlib.backends.backend_pdf import PdfPages
%matplotlib inline

In [52]:
def get_bigwig_mean(bw_file):
    import os
    import subprocess
    fold = subprocess.check_output(f'bigWigInfo {bw_file} | grep mean',
                                   shell=True).decode()
    fold = float(fold.split()[1])
    return fold

In [53]:
print('123')

123


# prepare

In [3]:
os.chdir("/mnt/Storage/home/wangyiman/CHMsInOtherContexts/CellStateTransition/MethK9Signal")

In [4]:
stage_dict = {
    'EarlyEmbryogenesis': ['2cell','8cell','Morula','ICM',],
    'PGCsDevelopment': ['E10.5','E13.5_female','E13.5_male',],
    'Spermatogenesis': ['US','DS','PS','RS',],
    'RetinalDevelopment': ['E14.5','E17.5','P0','P3','P7','P10','P14','P21',],
    'HeartDevelopment': ['E10.5','E11.5','E12.5','E13.5','E14.5','E15.5','E16.5','P0',],
    'LiverDevelopment': ['E11.5','E12.5','E13.5','E14.5','E15.5','E16.5','P0']
}

In [47]:
color_dict = {
    'EarlyEmbryogenesis': '#9B1C3D',
    'PGCsDevelopment': '#16557A',
    'Spermatogenesis': '#1B9E77',
    'RetinalDevelopment': '#BF8B12',
    'HeartDevelopment': '#2E8BC0',
    'LiverDevelopment': '#666666'
}

In [30]:
def hex2rgb(h) :
    rgb = tuple(int(h[i:i+2], 16) for i in (1, 3, 5))
    return f"{rgb[0]},{rgb[1]},{rgb[2]}"

In [31]:
hex2rgb('#9B1C3D')

'155,28,61'

In [28]:
smooth_dict = {
    'methyl.merged': 1,
    'H3K9me3.rmDup': 5
}

In [37]:
stage_dict.keys()

dict_keys(['EarlyEmbryogenesis', 'PGCsDevelopment', 'Spermatogenesis', 'RetinalDevelopment', 'HeartDevelopment', 'LiverDevelopment'])

In [38]:
%%bash
bedToBigBed Universal.CHM.bed /mnt/Storage/home/yanghui/annotations/mm10/mm10_euch.chrom.sizes Universal.CHM.bb

pass1 - making usageList (19 chroms): 1 millis
pass2 - checking and writing primary data (2677 records, 3 fields): 12 millis


# trackDb.txt

## DNA methyl signal & K9 signal

In [41]:
with open('trackDb.txt', 'w') as fi :
    for process in stage_dict.keys() :
        for modifi in ['methyl.merged', 'H3K9me3.rmDup'] :
            for stage in stage_dict[process] :
                bigwig_file = f'{process}_{stage}.{modifi}.bw'
                mean = get_bigwig_mean(f'{bigwig_file}')
                fi.write(f'track {process}_{stage}_{modifi.split(".")[0]}\n')
                fi.write(f'bigDataUrl {bigwig_file}\n')
                fi.write(f'shortLabel {process}_{stage}_{modifi.split(".")[0]}\n')
                fi.write(f'longLabel {process}_{stage}_{modifi.split(".")[0]}\n')
                fi.write(f'color {hex2rgb(color_dict[process])}\n')
                fi.write(f'type bigWig\n')
                fi.write(f'visibility full\n')
                if modifi == 'methyl.merged' :
                    fi.write(f'viewLimits 0:1\n')
                else :
#                     fi.write(f'viewLimits {5 * mean:.5f}:{50 * mean:.5f}\n')
                    fi.write(f'viewLimits 0:{10 * mean:.5f}\n')
#                     fi.write(f'viewLimits 0:1\n')
                fi.write(f'maxHeightPixels 15\n')
#                 fi.write(f'smoothingWindow {smooth_dict[modifi]}\n')
                fi.write(f'windowingFunction mean\n\n')
      
    fi.write('track CpG_number\n')
    fi.write('bigDataUrl mm10.b1kb.s10bp.euchr.CpGnumber.bw\n')    
    fi.write('shortLabel CpG_number\n')    
    fi.write('longLabel CpG_number\n')    
    fi.write('color 50,50,50\n')    
    fi.write('type bigWig\n')    
    fi.write('visibility full\n')    
    fi.write('maxHeightPixels 20\n\n')   
        
    fi.write('track Universal_CHMs\n')
    fi.write('bigDataUrl Universal.CHM.bb\n')    
    fi.write('shortLabel Universal_CHMs\n')    
    fi.write('longLabel Universal_CHMs\n')    
    fi.write('color 50,50,50\n')    
    fi.write('type bigBed\n')    
    fi.write('visibility full\n')    
    fi.write('maxHeightPixels 20\n\n')   
    

## DNA amount signal & K9 signal

In [54]:
with open('trackDb_DNAamount.txt', 'w') as fi :
    for p,process in enumerate(stage_dict.keys()) :
        for modifi in ['mm10.1kbB_10bpS.methyl_amount', 'H3K9me3.rmDup'] :
            for s,stage in enumerate(stage_dict[process]) :
#                 mean = get_bigwig_mean(f'{bigwig_file}')
                fi.write(f'track {process}_{stage}_{modifi.split(".")[-1]}\n')
                if modifi == 'mm10.1kbB_10bpS.methyl_amount' :
                    bigwig_file = f'{process}.{stage}.{modifi}.bw'
                else :
                    bigwig_file = f'{process}_{stage}.{modifi}.bw'
                fi.write(f'bigDataUrl {bigwig_file}\n')
                fi.write(f'shortLabel {process}_{stage}_{modifi.split(".")[-1]}\n')
                fi.write(f'longLabel {process}_{stage}_{modifi.split(".")[-1]}\n')
                fi.write(f'color {hex2rgb(color_dict[process])}\n')
                fi.write(f'type bigWig\n')
                fi.write(f'visibility full\n')
                if modifi == 'mm10.1kbB_10bpS.methyl_amount' :
                    fi.write(f'viewLimits 0:50\n')
                else :
#                     fi.write(f'viewLimits {5 * mean:.5f}:{50 * mean:.5f}\n')
                    fi.write(f'viewLimits 0:1\n')
#                     fi.write(f'viewLimits 0:1\n')
                fi.write(f'maxHeightPixels 20\n')
#                 fi.write(f'smoothingWindow {smooth_dict[modifi]}\n')
                fi.write(f'windowingFunction mean\n')
                process1 = process if process != 'EarlyEmbryogenesis' else 'Pre-implantation'
                fi.write(f'group {p+1}_{process1}\n')
                fi.write(f'prior {s+1}\n\n')
        
      
    fi.write('track CpG_number\n')
    fi.write('bigDataUrl mm10.b1kb.s10bp.euchr.CpGnumber.bw\n')    
    fi.write('shortLabel CpG_number\n')    
    fi.write('longLabel CpG_number\n')    
    fi.write('color 50,50,50\n')    
    fi.write('type bigWig\n')    
    fi.write('visibility full\n')    
    fi.write('maxHeightPixels 20\n\n')   
        
    fi.write('track Universal_CHMs\n')
    fi.write('bigDataUrl Universal.CHM.bb\n')    
    fi.write('shortLabel Universal_CHMs\n')    
    fi.write('longLabel Universal_CHMs\n')    
    fi.write('color 50,50,50\n')    
    fi.write('type bigBed\n')    
    fi.write('visibility full\n')    
    fi.write('maxHeightPixels 20\n\n')   
    

In [ ]:
track CR_Nanog_dome_rep1
bigDataUrl CR_Nanog_dome_rep1.bw
shortLabel CR_Nanog_dome_rep1
longLabel CR_Nanog_dome_rep1
color 152,189,216
type bigWig
visibility full
viewLimits 0.154665:1.546650
maxHeightPixels 20
smoothingWindow 5
windowingFunction mean

## A/B compartment

In [55]:
stage_3_dict = {
 'EarlyEmbryogenesis': ['2cell', '8cell', 'ICM'],
 'PGCsDevelopment': ['E10.5', 'E13.5_female', 'E13.5_male'],
 'Spermatogenesis': ['US', 'DS', 'RS'],
 'RetinalDevelopment': ['E14.5', 'P0', 'P21'],
 'HeartDevelopment': ['E10.5', 'E13.5', 'P0'],
 'LiverDevelopment': ['E11.5', 'E14.5', 'P0']
}

In [57]:
%%bash
ln -s /mnt/Storage/home/wangyiman/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/ChromatinStates/Cavalli_Cell2017_ABcompartment/AB_compartment_intersect[AB]_Cavalli_Cell2017.bed .

sort -k1,1 -k2,2n AB_compartment_intersectA_Cavalli_Cell2017.bed | grep -v chrX | grep -v chrY > A_comparment.sorted.bed
sort -k1,1 -k2,2n AB_compartment_intersectB_Cavalli_Cell2017.bed | grep -v chrX | grep -v chrY > B_comparment.sorted.bed
bedToBigBed A_comparment.sorted.bed /mnt/Storage/home/yanghui/annotations/mm10/mm10_euch.chrom.sizes A_comparment.bb
bedToBigBed B_comparment.sorted.bed /mnt/Storage/home/yanghui/annotations/mm10/mm10_euch.chrom.sizes B_comparment.bb


In [61]:
with open('trackDb_ABcompart.txt', 'w') as fi :
    for process in stage_dict.keys() :
        for modifi in ['methyl.merged', 'H3K9me3.rmDup'] :
            for stage in stage_3_dict[process] :
                bigwig_file = f'{process}_{stage}.{modifi}.bw'
                mean = get_bigwig_mean(f'{bigwig_file}')
                fi.write(f'track {process}_{stage}_{modifi.split(".")[0]}\n')
                fi.write(f'bigDataUrl {bigwig_file}\n')
                fi.write(f'shortLabel {process}_{stage}_{modifi.split(".")[0]}\n')
                fi.write(f'longLabel {process}_{stage}_{modifi.split(".")[0]}\n')
                fi.write(f'color {hex2rgb(color_dict[process])}\n')
                fi.write(f'type bigWig\n')
                fi.write(f'visibility full\n')
                if modifi == 'methyl.merged' :
                    fi.write(f'viewLimits 0.6:1\n')
                else :
#                     fi.write(f'viewLimits {5 * mean:.5f}:{50 * mean:.5f}\n')
#                     fi.write(f'viewLimits 0:{10 * mean:.5f}\n')
                    fi.write(f'viewLimits 0:1\n')
                fi.write(f'maxHeightPixels 15\n')
#                 fi.write(f'smoothingWindow {smooth_dict[modifi]}\n')
                fi.write(f'windowingFunction mean\n\n')
      
    fi.write('track CpG_number\n')
    fi.write('bigDataUrl mm10.b1kb.s10bp.euchr.CpGnumber.bw\n')    
    fi.write('shortLabel CpG_number\n')    
    fi.write('longLabel CpG_number\n')    
    fi.write('color 50,50,50\n')    
    fi.write('type bigWig\n')    
    fi.write('visibility full\n')    
    fi.write('viewLimits 0:50\n')
    fi.write('maxHeightPixels 20\n\n')   
        
    fi.write('track Universal_CHMs\n')
    fi.write('bigDataUrl Universal.CHM.bb\n')    
    fi.write('shortLabel Universal_CHMs\n')    
    fi.write('longLabel Universal_CHMs\n')    
    fi.write('color 50,50,50\n')    
    fi.write('type bigBed\n')    
    fi.write('visibility full\n')    
    fi.write('maxHeightPixels 20\n\n')
    
    fi.write('track A_compartments\n')
    fi.write('bigDataUrl A_comparment.bb\n')    
    fi.write('shortLabel A_compartments\n')    
    fi.write('longLabel A_compartments\n')    
    fi.write('color 196,90,198\n')    
    fi.write('type bigBed\n')    
    fi.write('visibility full\n')    
    fi.write('maxHeightPixels 20\n\n')
    
    fi.write('track B_compartments\n')
    fi.write('bigDataUrl B_comparment.bb\n')    
    fi.write('shortLabel B_compartments\n')    
    fi.write('longLabel B_compartments\n')    
    fi.write('color 90,173,198\n')    
    fi.write('type bigBed\n')    
    fi.write('visibility full\n')    
    fi.write('maxHeightPixels 20\n\n')

    

In [56]:
1

1